# Questions: 
    
- Using OneHotEncoder Columns with regresison
- Pipelines 

In [1]:
#read in file as dataframe 
# import pyspark modules
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, Row
from pyspark.ml.feature import OneHotEncoder, StringIndexer, Bucketizer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import pyspark.mllib.regression as reg
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.linalg import SparseVector, DenseVector
import functools 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("app") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '2') \
    .config('spark.cores.max', '2') \
    .config("spark.driver.memory",'4g') \
    .getOrCreate()

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

In [3]:
path_to_data = os.path.join("/home/jovyan/FlightDelay/clean_data_no_hot")

In [4]:
df = spark.read.format("csv") \
    .option("header", "true").option("inferschema","true").load(path_to_data)

In [5]:
#final_df.count()

In [5]:
pd_df = df.toPandas()

In [6]:
pd_df.head()

,_c0,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,B_SCHEDULED_ARRIVAL,B_ARRIVAL_TIME,B_SCHEDULED_DEPARTURE,B_DEPARTURE_TIME
0,0,2015,1,1,4,AA,2459,PHX,DFW,200,...,868,500,1476,4,0,1,1.0,4.0,0.0,4.0
1,1,2015,1,1,4,US,699,PHX,CLT,220,...,1773,804,728,-36,0,0,2.0,2.0,0.0,0.0
2,2,2015,1,1,4,DL,1890,DFW,ATL,545,...,731,849,847,-2,0,0,2.0,2.0,1.0,2.0
3,3,2015,1,1,4,OO,6457,SMX,LAX,545,...,134,651,1476,4,0,1,2.0,4.0,1.0,4.0
4,4,2015,1,1,4,OO,5160,OMA,IAH,550,...,781,810,831,21,0,0,2.0,2.0,1.0,2.0


# One Hot Encoder 

since vector type didn't transfer

In [7]:
# for each level, count freq. val=0 for most freq, then 1, ...

stringIndexer = StringIndexer(inputCol="AIRLINE", outputCol="AIRLINE_Index")
model = stringIndexer.fit(df)
indexed = model.transform(df)

encoder = OneHotEncoder(inputCol="AIRLINE_Index", outputCol="AIRLINE_Vec")
encoded = encoder.transform(indexed)


In [8]:
# for each level, count freq. val=0 for most freq, then 1, ...

stringIndexer2 = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="ORIGIN_AIRPORT_Index")
model2 = stringIndexer2.fit(encoded)
indexed2 = model2.transform(encoded)

encoder2 = OneHotEncoder(inputCol="ORIGIN_AIRPORT_Index", outputCol="ORIGIN_AIRPORT_Vec")
encoded2 = encoder2.transform(indexed2)



In [9]:
# for each level, count freq. val=0 for most freq, then 1, ...

stringIndexer3 = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DESTINATION_AIRPORT_Index")
model3 = stringIndexer3.fit(encoded2)
indexed3 = model3.transform(encoded2)

encoder3 = OneHotEncoder(inputCol="DESTINATION_AIRPORT_Index", outputCol="DESTINATION_AIRPORT_Vec")
encoded3 = encoder3.transform(indexed3)
encoded3.select('DESTINATION_AIRPORT','DESTINATION_AIRPORT_Index', "DESTINATION_AIRPORT_Vec").show()
#encoded3.cache()b

+-------------------+-------------------------+-----------------------+
|DESTINATION_AIRPORT|DESTINATION_AIRPORT_Index|DESTINATION_AIRPORT_Vec|
+-------------------+-------------------------+-----------------------+
|                DFW|                      2.0|        (585,[2],[1.0])|
|                CLT|                     14.0|       (585,[14],[1.0])|
|                ATL|                      0.0|        (585,[0],[1.0])|
|                LAX|                      3.0|        (585,[3],[1.0])|
|                IAH|                      5.0|        (585,[5],[1.0])|
|                DFW|                      2.0|        (585,[2],[1.0])|
|                DFW|                      2.0|        (585,[2],[1.0])|
|                DFW|                      2.0|        (585,[2],[1.0])|
|                FLL|                     22.0|       (585,[22],[1.0])|
|                DFW|                      2.0|        (585,[2],[1.0])|
|                MCO|                     10.0|       (585,[10],

In [10]:
new_cols_to_drop = ['AIRLINE_Index', 'AIRLINE', 'ORIGIN_AIRPORT_Index', 
                    'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT_Index',
                    'DESTINATION_AIRPORT', 'FLIGHT_NUMBER']

final_encoded = encoded3.drop(*new_cols_to_drop)

In [11]:
final_encoded.columns

['_c0',
 'YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'DISTANCE',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'B_SCHEDULED_ARRIVAL',
 'B_ARRIVAL_TIME',
 'B_SCHEDULED_DEPARTURE',
 'B_DEPARTURE_TIME',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec']

In [15]:
#final_df = final_encoded.withColumn('CANCELLED2', final_encoded.CANCELLED)

In [72]:
final_df = final_encoded.drop('_c0', 'YEAR', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'ARRIVAL_TIME', 
                              'B_ARRIVAL_TIME', 'ELASPED TIME', 'B_ARRIVAL_TIME', 
                              'B_DEPARTURE_TIME', 'DEPARTURE_TIME')

In [73]:
final_df.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- B_SCHEDULED_ARRIVAL: double (nullable = true)
 |-- B_SCHEDULED_DEPARTURE: double (nullable = true)
 |-- AIRLINE_Vec: vector (nullable = true)
 |-- ORIGIN_AIRPORT_Vec: vector (nullable = true)
 |-- DESTINATION_AIRPORT_Vec: vector (nullable = true)



# Scale Data

In [92]:
#convery final_encoded to rdd 
# we cannpot scale bucketized or vec columns, so we omit those form the scaling process
input_data = final_df.rdd.map(lambda x: (x[9], DenseVector(x[1:9])))

In [93]:
input_data.take(3)

[(1, DenseVector([1.0, 4.0, 200.0, 120.0, 137.0, 868.0, 500.0, 0.0])),
 (0, DenseVector([1.0, 4.0, 220.0, 224.0, 199.0, 1773.0, 804.0, 0.0])),
 (0, DenseVector([1.0, 4.0, 545.0, 124.0, 104.0, 731.0, 849.0, 0.0]))]

In [94]:
df2 = sqlCtx.createDataFrame(input_data, ["label","features2"])

In [95]:
SS = StandardScaler(inputCol = "features2", outputCol = "features_scaled")

scaler = SS.fit(df2)

In [96]:
#transform the data in df2 with our scaler 
scaled_df = scaler.transform(df2)
#join scalable feature with columns  'AIRLINE_Vec', 'ORIGIN_AIRPORT_Vec', 
#'DESTINATION_AIRPORT_Vec', 'B_SCHEDULED_ARRIVAL', 'B_ARRIVAL_TIME', 
#'B_SCHEDULED_DEPARTURE','B_DEPARTURE_TIME'


In [97]:
scaled_df.show(4)

+-----+--------------------+--------------------+
|label|           features2|     features_scaled|
+-----+--------------------+--------------------+
|    1|[1.0,4.0,200.0,12...|[0.11458541287952...|
|    0|[1.0,4.0,220.0,22...|[0.11458541287952...|
|    0|[1.0,4.0,545.0,12...|[0.11458541287952...|
|    1|[1.0,4.0,545.0,66...|[0.11458541287952...|
+-----+--------------------+--------------------+
only showing top 4 rows



In [98]:
# since there is no common column between these two dataframes add row_index so that it can be joined
scaled_df = scaled_df.withColumn('row_index', F.monotonically_increasing_id())
final_df = final_df.withColumn('row_index', F.monotonically_increasing_id())

In [99]:
# combine scaled df and final_df
total_df = scaled_df.join(final_df, scaled_df.row_index == final_df.row_index)

In [100]:
total_df.count()

100986

In [101]:
total_df.columns

['label',
 'features2',
 'features_scaled',
 'row_index',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'SCHEDULED_DEPARTURE',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'DISTANCE',
 'SCHEDULED_ARRIVAL',
 'DIVERTED',
 'CANCELLED',
 'B_SCHEDULED_ARRIVAL',
 'B_SCHEDULED_DEPARTURE',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec',
 'row_index']

In [102]:
#drop columns of already scaled predictors 
total_df = total_df.drop(*['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE',
                           'SCHEDULED_ARRIVAL', 'DIVERTED', 
                           'CANCELLED2', 'row_index']) 

In [103]:
total_df.count()

100986

In [104]:
#final scaled dataframe for predicting Cancelled flights 
total_df.columns

['label',
 'features2',
 'features_scaled',
 'ELAPSED_TIME',
 'CANCELLED',
 'B_SCHEDULED_ARRIVAL',
 'B_SCHEDULED_DEPARTURE',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec']

# Scaled: split data  

In [105]:
def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs)

In [106]:
# create scaled train/test with one-hot-encoded columns
train_data_scaled, test_data_scaled = total_df.randomSplit(seed= 314, weights= [.80, 0.20])

In [107]:
# create scaled train/test without one-hot-encoded columns 
train_data_scaled_no_hot, test_data_scaled_no_hot = scaled_df.randomSplit(seed = 314, weights = [0.80, 0.20])

In [108]:
# even out scaled dataframe with no one hot encoded
c_no_hot = train_data_scaled_no_hot.where(scaled_df.label ==1)
not_c_no_hot = train_data_scaled_no_hot.where(scaled_df.label ==0).sample(False, 0.025, 454)

In [109]:
c_no_hot.count()

1295

In [110]:
not_c_no_hot.count()

2007

In [111]:
# union scaled training data no hot 
training_data_scaled_no_hot = unionAll([c_no_hot, not_c_no_hot])
training_data_scaled_no_hot.show(3)

+-----+--------------------+--------------------+---------+
|label|           features2|     features_scaled|row_index|
+-----+--------------------+--------------------+---------+
|    1|[1.0,1.0,805.0,20...|[0.11458541287952...|    45566|
|    1|[1.0,1.0,1200.0,7...|[0.11458541287952...|    45639|
|    1|[1.0,1.0,1552.0,9...|[0.11458541287952...|    45700|
+-----+--------------------+--------------------+---------+
only showing top 3 rows



In [34]:
c = train_data_scaled.where(total_df.label == 1)

In [35]:
## How to filter out large portion of non-canclelled flights in training data to make data more even 
not_c = train_data_scaled.where(total_df.label == 0).sample(False, .025, 454) 

In [36]:
not_c.count()

2007

In [37]:
c.count()

1295

In [38]:
training_data_scaled = unionAll([c, not_c])
training_data_scaled.show(4) 

+-----+--------------------+--------------------+---------+-------------------+--------------+---------------------+----------------+--------------+------------------+-----------------------+
|label|           features2|     features_scaled|CANCELLED|B_SCHEDULED_ARRIVAL|B_ARRIVAL_TIME|B_SCHEDULED_DEPARTURE|B_DEPARTURE_TIME|   AIRLINE_Vec|ORIGIN_AIRPORT_Vec|DESTINATION_AIRPORT_Vec|
+-----+--------------------+--------------------+---------+-------------------+--------------+---------------------+----------------+--------------+------------------+-----------------------+
|    1|[1.0,1.0,805.0,13...|[0.11458541287952...|        1|                4.0|           4.0|                  2.0|             4.0|(13,[5],[1.0])|   (593,[4],[1.0])|       (585,[41],[1.0])|
|    1|[1.0,1.0,1200.0,1...|[0.11458541287952...|        1|                4.0|           4.0|                  4.0|             4.0|(13,[8],[1.0])|  (593,[17],[1.0])|       (585,[13],[1.0])|
|    1|[1.0,1.0,1552.0,1...|[0.114585412

In [39]:
training_data_scaled.columns

['label',
 'features2',
 'features_scaled',
 'CANCELLED',
 'B_SCHEDULED_ARRIVAL',
 'B_ARRIVAL_TIME',
 'B_SCHEDULED_DEPARTURE',
 'B_DEPARTURE_TIME',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec']

In [40]:
training_data_scaled_no_hot.count()

3302

# Un-scaled: Split Data

In [113]:
train_data, test_data = final_df.randomSplit(seed= 314, weights= [.80, 0.20])

In [59]:
c2 = train_data.where(final_df.CANCELLED == 1)
not_c2 = train_data.where(final_df.CANCELLED == 0).sample(False, 0.02, 99)

In [60]:
c2.count()

1312

In [61]:
not_c2.count()

1603

In [62]:
training_data = unionAll([c2, not_c2])

In [63]:
training_data.count()

2915

In [64]:
training_data.groupBy('CANCELLED').count().show()

+---------+-----+
|CANCELLED|count|
+---------+-----+
|        1| 1312|
|        0| 1603|
+---------+-----+



# Gradient Boosted Trees

In [114]:
vars_to_keep = [
 'features_scaled',
 'B_SCHEDULED_ARRIVAL',
 'B_SCHEDULED_DEPARTURE',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec'
]

In [115]:
training_data_scaled.columns

['label',
 'features2',
 'features_scaled',
 'CANCELLED',
 'B_SCHEDULED_ARRIVAL',
 'B_ARRIVAL_TIME',
 'B_SCHEDULED_DEPARTURE',
 'B_DEPARTURE_TIME',
 'AIRLINE_Vec',
 'ORIGIN_AIRPORT_Vec',
 'DESTINATION_AIRPORT_Vec']

In [116]:
training_data_scaled_no_hot.show(2)

+-----+--------------------+--------------------+---------+
|label|           features2|     features_scaled|row_index|
+-----+--------------------+--------------------+---------+
|    1|[1.0,1.0,805.0,20...|[0.11458541287952...|    45566|
|    1|[1.0,1.0,1200.0,7...|[0.11458541287952...|    45639|
+-----+--------------------+--------------------+---------+
only showing top 2 rows



In [117]:
# class = 2 for benign (negative class, 4 for malignant (positive class)
target = 'label'
positive_label = 1
negative_label = 0

SEED = 314
ITERS = 10
FOLDS = 5

In [119]:
assembler = VectorAssembler(
                            inputCols=[c for c in vars_to_keep],
                            outputCol='features')

In [120]:
train = (assembler.transform(training_data_scaled).select(target, "features"))

In [121]:
train

DataFrame[label: bigint, features: vector]

In [122]:
train.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(1205,[0,1,2,3,4,...|
|    1|(1205,[0,1,2,3,4,...|
|    1|(1205,[0,1,2,3,4,...|
|    1|(1205,[0,1,2,3,4,...|
|    1|(1205,[0,1,2,3,4,...|
+-----+--------------------+
only showing top 5 rows



In [123]:
gbt = GBTClassifier(labelCol=target, featuresCol="features", maxIter=ITERS)

In [124]:
evaluator = BinaryClassificationEvaluator(labelCol=target)

In [125]:
paramGrid = ParamGridBuilder() \
            .addGrid(gbt.maxIter, [1, 10]) \
            .addGrid(gbt.maxDepth, [1, 2]) \
            .build()

In [126]:
# k-fold cross validation
crossval = CrossValidator(
            estimator=gbt, 
            estimatorParamMaps=paramGrid, 
            evaluator=evaluator, 
            numFolds=FOLDS)

In [ ]:
# part that returns error
model = crossval.fit(train)

In [ ]:
model

In [99]:
predictions = model.transform(assembler.transform(test_data_scaled.select(vars_to_keep)).select("features"))

In [100]:
test_data_scaled.show(2)

+-----+--------------------+--------------------+---------+-------------------+--------------+---------------------+----------------+--------------+------------------+-----------------------+---------+
|label|           features2|     features_scaled|CANCELLED|B_SCHEDULED_ARRIVAL|B_ARRIVAL_TIME|B_SCHEDULED_DEPARTURE|B_DEPARTURE_TIME|   AIRLINE_Vec|ORIGIN_AIRPORT_Vec|DESTINATION_AIRPORT_Vec|row_index|
+-----+--------------------+--------------------+---------+-------------------+--------------+---------------------+----------------+--------------+------------------+-----------------------+---------+
|    0|[1.0,1.0,505.0,50...|[0.11458541287952...|        0|                3.0|           3.0|                  1.0|             1.0|(13,[2],[1.0])|   (593,[6],[1.0])|        (585,[2],[1.0])|        0|
|    0|[1.0,1.0,535.0,54...|[0.11458541287952...|        0|                2.0|           2.0|                  1.0|             1.0|(13,[4],[1.0])|  (593,[40],[1.0])|        (585,[1],[1.0])| 

Convert the dataframe to an rdd. Then select only the prediction and feature fields

In [101]:
predictions.show(5)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(1207,[0,1,2,3,4,...|[1.32590267922033...|[0.93412217565278...|       0.0|
|(1207,[0,1,2,3,4,...|[1.67697666691614...|[0.96623405454987...|       0.0|
|(1207,[0,1,2,3,4,...|[1.32590267922033...|[0.93412217565278...|       0.0|
|(1207,[0,1,2,3,4,...|[1.32590267922033...|[0.93412217565278...|       0.0|
|(1207,[0,1,2,3,4,...|[1.21914704037651...|[0.91970119430999...|       0.0|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [102]:
predictions = predictions.withColumn('row_index', F.monotonically_increasing_id())
test_data_scaled = test_data_scaled.withColumn('row_index', F.monotonically_increasing_id())

In [103]:
# combine scaled df and final_df
tot_preds = predictions.join(test_data_scaled, predictions.row_index == test_data_scaled.row_index)

# Calculate Test Error for GBT method 

In [104]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="CANCELLED", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(tot_preds)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.011143


In [105]:
feat_imp = DenseVector(model.bestModel.featureImportances)

In [106]:
model.bestModel

GBTClassificationModel (uid=GBTClassifier_d5e32419c9f6) with 10 trees

In [ ]:
# This part doesn't seem to work. 
feat_imp = DenseVector(model.bestModel.featureImportances); feat_imp

In [160]:
# pack variables, feature importances into pandas dataframe
#df_feat_imp = pd.DataFrame(index=vars_to_keep2, columns=['feature_importance'], data=feat_imp[:])
#df_feat_imp.sort_values(by='feature_importance', ascending=False, inplace=True)

In [159]:
#df_feat_imp

The important variables in predicting wether a flight was cancelled are: Departure_Delay and Arrival_delay

# Random Forest

Add cross validation to this model 

In [117]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="CANCELLED", outputCol="indexedLabel").fit(final_df)

In [118]:
assembler = VectorAssembler(
                            inputCols=[c for c in vars_to_keep],
                            outputCol='features')
data = (assembler.transform(total_df).select(target, "features"))


In [119]:
scaled_df.show(3)

+-----+--------------------+--------------------+---------+
|label|           features2|     features_scaled|row_index|
+-----+--------------------+--------------------+---------+
|    1|[1.0,4.0,200.0,13...|[0.11458541287952...|        0|
|    0|[1.0,4.0,220.0,20...|[0.11458541287952...|        1|
|    0|[1.0,4.0,545.0,60...|[0.11458541287952...|        2|
+-----+--------------------+--------------------+---------+
only showing top 3 rows



In [133]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

In [134]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed = 314)

In [135]:
trainingData.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
+-----+--------------------+
only showing top 5 rows



In [136]:
c3 = trainingData.where(data.label == 1)
not_c3 = trainingData.where(data.label == 0).sample(False, 0.025, 99)

trainingData2 = unionAll([c3, not_c3])

In [132]:
testData.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
|    0|(1207,[0,1,2,3,4,...|
+-----+--------------------+
only showing top 3 rows



In [145]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="indexedFeatures", numTrees=10)

In [146]:
pipeline = Pipeline(stages=[featureIndexer, rf])

In [127]:
# Convert indexed labels back to original labels.
#labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

In [150]:
# class = 2 for benign (negative class, 4 for malignant (positive class)
target = 'label'
positive_label = 1
negative_label = 0

SEED = 314
ITERS = 10
FOLDS = 5

In [151]:
evaluator = BinaryClassificationEvaluator(labelCol=target)


In [152]:
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=FOLDS)

In [153]:
# Train model.  This also runs the indexers.
model = crossval.fit(trainingData2)

In [154]:
# Make predictions.
predictions = model.transform(testData)

In [156]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(1207,[0,1,2,3,4,...|
|       0.0|    0|(1207,[0,1,2,3,4,...|
|       0.0|    0|(1207,[0,1,2,3,4,...|
|       0.0|    0|(1207,[0,1,2,3,4,...|
|       0.0|    0|(1207,[0,1,2,3,4,...|
+----------+-----+--------------------+
only showing top 5 rows



In [158]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0111926


In [182]:

print(rfModel)  # summary only

RandomForestClassificationModel (uid=RandomForestClassifier_ac9d9f81e22a) with 10 trees
